# AdaptiveGraph Interactive Demo

This notebook demonstrates how to use `adaptivegraph` with `langgraph` to create a self-optimizing workflow.

We will build a simple routing agent that learns to send "VIP" users to a premium model and "Guest" users to a fast model.

In [ ]:
# Install local package in editable mode with dev dependencies
# This installs 'adaptivegraph' along with 'matplotlib' and other dev tools defined in pyproject.toml
# NOTE: We use '..' because this notebook is inside the 'notebooks/' directory
%pip install -e "..[all]"

In [1]:
import random
import matplotlib.pyplot as plt
from typing import Literal, TypedDict
from langgraph.graph import StateGraph, END
from adaptivegraph import LearnableEdge

# Define our state
class AgentState(TypedDict):
    user_type: str
    query: str
    path_taken: str
    outcome: float

In [2]:
# Define Nodes

def start_node(state: AgentState):
    print(f"Processing request for {state['user_type']}...")
    return state

def premium_model(state: AgentState):
    return {"path_taken": "premium", "outcome": 1.0 if state["user_type"] == "vip" else 0.0}

def fast_model(state: AgentState):
    return {"path_taken": "fast", "outcome": 1.0 if state["user_type"] == "guest" else 0.0}

In [3]:
# Define the Learnable Edge

router = LearnableEdge(
    options=["premium", "fast"],
    policy="linucb",
    feature_dim=16,
    exploration_alpha=0.5,
    value_key="user_type" # Extract this key from state automatically
)
# No wrapper needed anymore!

In [4]:
# Build the Graph
workflow = StateGraph(AgentState)

workflow.add_node("start", start_node)
workflow.add_node("premium", premium_model)
workflow.add_node("fast", fast_model)

workflow.set_entry_point("start")

# Add conditional edge directly!
workflow.add_conditional_edges(
    "start",
    router,
    {
        "premium": "premium",
        "fast": "fast"
    }
)

workflow.add_edge("premium", END)
workflow.add_edge("fast", END)

app = workflow.compile()

In [5]:
# Simulation Loop
history = []
accuracies = []

for i in range(100):
    # Generate synthetic data
    u_type = "vip" if random.random() < 0.5 else "guest"
    initial_state = {"user_type": u_type, "query": "hello", "path_taken": "", "outcome": 0.0}
    
    # Run Graph
    result = app.invoke(initial_state)
    
    # Feedback Loop
    # In this toy example, the nodes themselves calculated the 'outcome' (reward)
    reward = result["outcome"]
    
    # CRITICAL: Teach the router!
    router.record_feedback(result, reward=reward)
    
    history.append(reward)
    avg_acc = sum(history[-20:]) / len(history[-20:])
    accuracies.append(avg_acc)
    
    if i % 10 == 0:
        print(f"Step {i}: Type={u_type}, Path={result['path_taken']}, Reward={reward}")

print(f"Final Accuracy (last 20): {accuracies[-1]:.2f}")

In [6]:
# Plot Learning Curve
plt.plot(accuracies)
plt.title("Routing Accuracy over Time")
plt.xlabel("Iterations")
plt.ylabel("Moving Average Accuracy")
plt.ylim(0, 1.1)
plt.show()

## Advanced: Semantic Routing (Batteries Included)

In this section, we will use `SentenceTransformer` to route requests based on their **semantic meaning** (text content) rather than a simple categorical tag.

We will build a help-desk graph that routes queries to: `technical_support`, `billing`, or `general_chat`.

In [4]:
# 1. Create the Semantic Router
try:
    # value_key="query" tells the edge to look at state["query"] for embedding
    semantic_edge = LearnableEdge.create(
        options=["technical_support", "billing", "general_chat"],
        embedding="sentence-transformers",
        memory="faiss",
        feature_dim=384, 
        exploration_alpha=0.2,
        value_key="query"
    )
    print("Semantic Edge created successfully!")
except ImportError:
    print("Please install 'sentence-transformers' and 'faiss-cpu' to run this section.")

Semantic Edge created successfully!


In [5]:
# 2. Define Nodes and Graph

class HelpState(TypedDict):
    query: str
    response: str

def tech_node(state):
    return {"response": "Connecting you to an engineer..."}

def billing_node(state):
    return {"response": "Opening invoice portal..."}

def chat_node(state):
    return {"response": "I can help with general questions!"}

graph = StateGraph(HelpState)
graph.add_node("start", lambda x: x) # Pass through
graph.add_node("technical_support", tech_node)
graph.add_node("billing", billing_node)
graph.add_node("general_chat", chat_node)

graph.set_entry_point("start")

# Use semantic_edge to route from start
graph.add_conditional_edges(
    "start",
    semantic_edge,
    {
        "technical_support": "technical_support",
        "billing": "billing",
        "general_chat": "general_chat"
    }
)
graph.add_edge("technical_support", END)
graph.add_edge("billing", END)
graph.add_edge("general_chat", END)

help_desk_app = graph.compile()

In [8]:
# 3. Train the Semantic Router
# We need to provide feedback so it learns which sentences correspond to which node.

train_data = [
    ("My screen is black", "technical_support"),
    ("Where is my invoice?", "billing"),
    ("Hello there", "general_chat"),
    ("Python script error", "technical_support"),
    ("I want a refund", "billing"),
    ("System crash", "technical_support")
]

print("Training interactive graph...")

for i in range(50):
    query, expected_node = random.choice(train_data)
    
    # Run the graph
    initial = {"query": query, "response": ""}
    # Note: In a real app, you would inspect the trace or return the node name to know where it went.
    # Here, we use the edge directly to 'record_feedback' because we need to know what IT predicted.
    # But let's verify via the output messages.
    
    # For training the bandit properly, we usually need the action it took.
    # The 'semantic_edge' object stores the last action it performed internally.
    
    result = help_desk_app.invoke(initial)
    resp = result["response"]
    
    # Infer path taken from response (just for this demo)
    path_taken = ""
    if "engineer" in resp: path_taken = "technical_support"
    elif "invoice" in resp: path_taken = "billing"
    else: path_taken = "general_chat"

    # Reward
    reward = 1.0 if path_taken == expected_node else -0.5
    
    # Teach the edge
    semantic_edge.record_feedback(result, reward)
    
    if i % 5 == 0:
        print(f"'{query}' -> {path_taken} (Reward: {reward})")

print("Training Complete.")

Training interactive graph...
'Where is my invoice?' -> billing (Reward: 1.0)
'System crash' -> technical_support (Reward: 1.0)
'My screen is black' -> technical_support (Reward: 1.0)
'I want a refund' -> billing (Reward: 1.0)
'I want a refund' -> billing (Reward: 1.0)
'Python script error' -> technical_support (Reward: 1.0)
'Python script error' -> technical_support (Reward: 1.0)
'Where is my invoice?' -> billing (Reward: 1.0)
'Python script error' -> technical_support (Reward: 1.0)
'Hello there' -> general_chat (Reward: 1.0)
Training Complete.


In [ ]:
# 4. Test on NEW queries (Generalization)
# The router should understand these concepts even if it hasn't seen the exact words.

test_queries = [
    "My payment failed",        # Should be Billing
    "Error 404 on the website", # Should be Tech
    "Good morning team"         # Should be Chat
]x

print("\n--- Generalization Test ---")
for q in test_queries:
    res = help_desk_app.invoke({"query": q, "response": ""})
    print(f"Q: {q.ljust(30)} -> A: {res['response']}")


--- Generalization Test ---
Q: My payment failed              -> A: Opening invoice portal...
Q: Error 404 on the website       -> A: Connecting you to an engineer...
Q: Good morning team              -> A: I can help with general questions!
